In [ ]:
!nvidia-smi

Sat May  3 17:26:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install datasets
!pip install trl==0.9.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import huggingface_hub
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    Trainer,
    TrainingArguments,)
import torch
from textwrap import dedent
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer
from torch.utils.data import DataLoader
from peft import (
    LoraConfig,
    PeftModel,
    PeftConfig,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training,
)

In [ ]:
# WILL NEED PERMISSION FROM META ON HF
# AND API KEY TO LOGIN
huggingface_hub.login()

In [ ]:
TRAIN_CSV = 
VAL_CSV = 
TEST_CSV = 
BASE_MODEL = 'meta-llama/Llama-3.2-1B-Instruct'
FINETUNED_MODEL = 
OUTPUT_DIR = 
PAD_TOKEN = '<|pad|>'

## Understanding the Tokenizer

In [106]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tokenizer.add_special_tokens({'pad_token': PAD_TOKEN})
tokenizer.padding_side = 'right'   #its the default

In [107]:
print('================== TOKENIZER DETAILS ==================')
print(f'All the special tokens - {tokenizer.all_special_tokens}')
print(tokenizer.bos_token, tokenizer.bos_token_id)
print(tokenizer.eos_token, tokenizer.eos_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)
print(f'cofig pad token - {tokenizer.convert_tokens_to_ids(PAD_TOKEN)}')
print('=======================================================')

================== TOKENIZER DETAILS ==================
All the special tokens - ['<|begin_of_text|>', '<|eot_id|>', '<|pad|>']
<|begin_of_text|> 128000
<|eot_id|> 128009
<|pad|> 128256
cofig pad token - 128256


## Training Prompts

In [114]:
def train_prompt(row):
    prompt = dedent(
        f"""
    {row["question"]}

    Information:

    ```
    {row["context"]}
    ```
    """
    )
    messages = [
        {
            "role": "system",
            "content": "Use the information to answer the question",
        },
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": row["answer"]},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)


## Loading the dataset

In [111]:
train_df = pd.read_csv(TRAIN_CSV)
val_df = pd.read_csv(VAL_CSV)
test_df = pd.read_csv(TEST_CSV)

In [112]:
train_df['model_input'] = train_df.apply(train_prompt, axis = 1)
train_df.head(7)

,question,answer,context,source,subdomain,model_input
0,What role do faculty play in the educational e...,Faculty are expert and contribute to innovativ...,"Chemical Engineering World renowned faculty, s...",http://cpe.engr.ku.edu/research-centers,cpe.engr.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
1,In which fields does Michael Branicky apply hi...,Michael Branicky applies his research in field...,These systems are applied to a wide range of a...,https://engr.ku.edu/news/article/engineering-p...,engr.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
2,What aspects of signal processing are included...,Aspects include design and implementation of d...,Signals and Systems includes signal processing...,https://eecs.ku.edu/phd-qualifying-exam-topics,eecs.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
3,What kind of communication role does Cody Howa...,He communicates news of School of Engineering ...,Communicates news of School of Engineering ach...,https://engr.ku.edu/people/cody-howard,engr.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
4,What specific classes are allowed to have alte...,Alternate examination times for MATH 2 and MAT...,Alternate examination times for MATH 2 and MAT...,https://registrar.ku.edu/fall-2025-final-exam-...,registrar.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
5,How much total scholarship money is awarded to...,"$12,000 in scholarships will be awarded to the...","A total of $12,000 in scholarships will be awa...",https://engr.ku.edu/high-school-design-competi...,engr.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
6,What is the main reason the University of Kans...,The University of Kansas does not write an inv...,The University of Kansas will not write an inv...,https://iss.ku.edu/invitation-letter-family,iss.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...


In [113]:
val_df['model_input'] = val_df.apply(train_prompt, axis = 1)
val_df.head(7)

,question,answer,context,source,subdomain,model_input
0,What is one of the expectations for Bioenginee...,Students are expected to maintain a 3.0 GPA in...,Students are expected to: Maintain a 3.0 GPA i...,http://bio.engr.ku.edu/overview,bio.engr.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
1,What are the specific roles of Emma Scott at t...,Emma Scott is the Self Engineering Leadership ...,Emma Scott Self Engineering Leadership Fellows...,https://engr.ku.edu/people/emma-scott,engr.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
2,What motivated Elizabeth Appel to pursue a Mas...,Appel felt there was a gap in her understandin...,"""I felt there was a gap in my understanding of...",https://engr.ku.edu/news/article/ku-senior-fro...,engr.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
3,How many students are expected to attend the K...,"More than 1,800 students are registered to att...","More than 1,800 students are registered to att...",https://engr.ku.edu/news/article/expo-showcase...,engr.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
4,What are the program objectives concerning sta...,The program aims to understand how to design a...,Understand how to design appropriate static/ad...,https://eecs.ku.edu/signal-processing,eecs.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
5,What is the eligibility requirement for kinder...,Children who will be at least 5 years old by A...,Children who will be at least 5 years old by A...,https://iss.ku.edu/school-enrollment-children,iss.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...
6,What should students do regarding their travel...,Students should inform the ISS Insurance Team ...,Please let the ISS Insurance Team know your tr...,https://iss.ku.edu/out-country-waiver,iss.ku.edu,<|begin_of_text|><|start_header_id|>system<|en...


In [ ]:
qa_train = Dataset.from_pandas(train_df)
qa_val = Dataset.from_pandas(val_df)
qa_test = Dataset.from_pandas(test_df)
print(qa_train)
print(qa_val)
print(qa_test)

Dataset({
    features: ['question', 'answer', 'context', 'source', 'model_input', 'num_tokens', 'subdomain', '__index_level_0__'],
    num_rows: 3066
})
Dataset({
    features: ['question', 'answer', 'context', 'source', 'model_input', 'num_tokens', 'subdomain', '__index_level_0__'],
    num_rows: 438
})
Dataset({
    features: ['question', 'answer', 'context', 'source', 'model_input', 'num_tokens', 'subdomain', '__index_level_0__'],
    num_rows: 876
})


## Understanding the model

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quantization_config,
    device_map="auto",
)
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128264, 2048)

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128264, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

## Training the model

In [ ]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 22,544,384 || all params: 1,258,375,168 || trainable%: 1.7915


In [ ]:
sft_config = SFTConfig(
    output_dir = OUTPUT_DIR,
    dataset_text_field="model_input",
    # max_seq_length=512,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    eval_strategy="steps",
    eval_steps=0.2,
    save_steps=0.2,
    logging_steps=10,
    learning_rate=1e-4,
    fp16=True,  # or bf16=True,
    save_strategy="steps",
    warmup_ratio=0.1,
    save_total_limit=2,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    save_safetensors=True,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False,  # No need to add additional separator token
    },
    seed=42,
)

response_template = "<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=qa_train,
    eval_dataset=qa_val,
    tokenizer=tokenizer,
    data_collator=collator,
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3066 [00:00<?, ? examples/s]

Map:   0%|          | 0/438 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:402: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
batch = next(iter(trainer.get_train_dataloader()))
print(batch)
print(tokenizer.decode([id for id in batch["input_ids"][0] if id != -100]))
print(tokenizer.decode([id for id in batch["labels"][0] if id != -100]))

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   2839,   3297,    220,   2366,     20,    271,  10464,    279,
           2038,    311,   4320,    279,   3488, 128009, 128006,    882, 128007,
            271,  15546,    374,    279,  10783,    315,  69102,   5383,  38716,
           1980,  15218,   1473,  14196,   4077,  33697,    580,  48290,    482,
          38294,     11,  69102,   5383,  38716,    482,     43,    440,   7889,
             11,   6150,    315,   8184,    198,  74694, 128009, 128006,  78191,
         128007,    271,  33697,    580,  48290,  10187,    279,   2361,    315,
          10783,    315,  69102,   5383,  38716,     13, 128009, 128256, 128256,
         128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256,
         128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256,
         12825

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
77,0.518700,0.520282
154,0.432600,0.506516
231,0.435500,0.492678
308,0.489400,0.493618


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_r

TrainOutput(global_step=383, training_loss=0.48783454820319194, metrics={'train_runtime': 1031.5758, 'train_samples_per_second': 2.972, 'train_steps_per_second': 0.371, 'total_flos': 2326988430581760.0, 'train_loss': 0.48783454820319194, 'epoch': 0.9993476842791911})

In [ ]:
trainer.save_model(FINETUNED_MODEL)

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


## Model Inferences

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
)

model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)
model = PeftModel.from_pretrained(model, FINETUNED_MODEL)
model = model.merge_and_unload()

In [ ]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False,
)

Device set to use cuda:0


In [ ]:
%%time
# 125 50
row = qa_test[100]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

When is the CR/No CR deadline for courses that last less than five days?

Information:

```
The CR/No CR deadline of courses that last less than five days is the day before the first day of the class.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     The CR/No CR deadline for these courses is the day before the first day of the class.
Finetuned model answer: The CR/No CR deadline for courses that last less than five days is the day before the first day of the class.

CPU times: user 712 ms, sys: 5.56 ms, total: 718 ms
Wall time: 2.01 s


In [ ]:

%%time
row = qa_test[123]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What topics are covered in the book regarding research methodologies?

Information:

```
It delves into the intricacies of HRI methodologies, statistical measurements, and research design, providing clear explanations and real-world examples for deeper understanding.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     The book delves into the intricacies of HRI methodologies, statistical measurements, and research design.
Finetuned model answer: The book delves into the intricacies of HRI methodologies, statistical measurements, and research design, providing clear explanations and real-world examples for deeper understanding.

CPU times: user 1.07 s, sys: 0 ns, total: 1.07 s
Wall time: 1.56 s


In [ ]:
row = qa_test[23]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

In which city and state is Sarah Zey's office located?

Information:

```
Lawrence, KS 66045
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Sarah Zey's office is located in Lawrence, KS.
Finetuned model answer: Lawrence, KS



In [ ]:
row = qa_test[98]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the purpose of the Bioengineering Opportunity Fund at the University of Kansas?

Information:

```
Bioengineering Opportunity Fund: Unrestricted funds that allow the Director to meet urgent needs and take advantage of opportunities as they arise.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     The Bioengineering Opportunity Fund provides unrestricted funds that allow the Director to meet urgent needs and take advantage of opportunities as they arise.
Finetuned model answer: The Bioengineering Opportunity Fund is unrestricted funds that allow the Director to meet urgent needs and take advantage of opportunities as they arise.



In [ ]:
row = qa_test[756]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What are the five critical areas of holistic financial wellness emphasized by Jayhawk Finances?

Information:

```
Jayhawk Finances provides coaching sessions, workshops and information about the five critical areas of holistic financial wellness: Earning Spending Borrowing Protecting Saving and Investing Finances.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     The five critical areas of holistic financial wellness are Earning, Spending, Borrowing, Protecting, Saving, and Investing Finances.
Finetuned model answer: The five critical areas of holistic financial wellness emphasized by Jayhawk Finances are Earning, Borrowing, Protecting, Saving, and Investing.



In [ ]:
row = qa_test[90]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

How do Graduate Scholarships differ from Graduate Fellowships at the University of Kansas?

Information:

```
Graduate Scholarships: Scholarships can be added onto fellowships, graduate teaching assistantships or graduate research assistantships, but are only offered for one year.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Graduate Scholarships can be added onto fellowships or assistantships but are only offered for one year, whereas fellowships can provide support for one or more years.
Finetuned model answer: Graduate Scholarships can be added onto fellowships, graduate teaching assistantships or graduate research assistantships but are only offered for one year.



In [ ]:
%%time
# 125 50
row = qa_test[765]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What specific area of cybersecurity is Koyel Pramanick focusing on in her PhD research?

Information:

```
Koyel is currently a PhD candidate in Computer Science. She is working in the field of cybersecurity, more specifically in software security.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Koyel is focusing on software security.
Finetuned model answer: She is focusing on software security.

CPU times: user 276 ms, sys: 0 ns, total: 276 ms
Wall time: 368 ms


In [ ]:
%%time
# 125 50
row = qa_test[403]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What must you do with your I-94 record after finding it?

Information:

```
Once you find your I-94 record, please upload it to iHawk on the 'ISS Check-in' tab.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     You must upload it to iHawk on the 'ISS Check-in' tab after finding your I-94 record.
Finetuned model answer: You must upload it to iHawk on the 'ISS Check-in' tab.

CPU times: user 574 ms, sys: 1.06 ms, total: 575 ms
Wall time: 615 ms


In [ ]:
%%time
# 125 50
row = qa_test[450]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What happens if a student does not check the box for the Bioengineering Certificate when applying for their degree?

Information:

```
When you apply for your degree, make sure to CHECK the BOX for the Bioengineering Certificate! Otherwise, we assume you don't want it.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     It is assumed that the student does not want the Bioengineering Certificate.
Finetuned model answer: If a student does not check the box for the Bioengineering Certificate, they assume they do not want it.

CPU times: user 655 ms, sys: 1.57 ms, total: 657 ms
Wall time: 1.29 s


In [ ]:
%%time
# 125 50
row = qa_test[340]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What storage capacity is provided in both laptops and desktops at the School of Engineering?

Information:

```
...Storage: 1TB SSD... Storage: 1TB SSD
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Both laptops and desktops have a storage capacity of 1TB SSD.
Finetuned model answer: Both laptops and desktops are equipped with a 1TB SSD.

CPU times: user 357 ms, sys: 0 ns, total: 357 ms
Wall time: 357 ms


In [ ]:
%%time
# 125 50
row = qa_test[203]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the email address for contacting Tim Miller?

Information:

```
Contact Info tim.j.miller@ku.edu
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Tim Miller's email address is tim.j.miller@ku.edu.
Finetuned model answer: The email address for Tim Miller is tim.j.miller@ku.edu.

CPU times: user 530 ms, sys: 0 ns, total: 530 ms
Wall time: 694 ms


In [ ]:
%%time
# 125 50
row = qa_test[506]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

Who should be contacted for questions regarding the Distinguished Engineering Service Award nomination process?

Information:

```
Contact Amy Wierman , 785-864-2930 , in the Engineering Deans Office with questions.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Amy Wierman should be contacted for questions regarding the nomination process.
Finetuned model answer: Amy Wierman, in the Engineering Deans Office.

CPU times: user 430 ms, sys: 780 µs, total: 431 ms
Wall time: 440 ms


In [ ]:
%%time
# 125 50
row = qa_test[120]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What role does Candan Tamerler hold within the administration related to research?

Information:

```
Candan Tamerler, Ph.D. -Track Director, Biomaterials & Tissue Engineering -Wesley G. Cramer Professor, Mechanical Engineering -Associate Vice Chancellor for Research...
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Candan Tamerler, Ph.D. is the Associate Vice Chancellor for Research.
Finetuned model answer: He is the Track Director of Biomaterials & Tissue Engineering and the Wesley G. Cramer Professor of Mechanical Engineering.

CPU times: user 605 ms, sys: 0 ns, total: 605 ms
Wall time: 605 ms


In [ ]:
%%time
# 125 50
row = qa_test[795]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What role do faculty members play in the education of students in the chemical and petroleum engineering department?

Information:

```
Chemical Engineering World renowned faculty, state-of-the-art facilities, innovative teaching and research prepare students to solve problems and positively impact the world.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Faculty members are expert educators who teach using innovative methods to prepare students for real-world challenges.
Finetuned model answer: Faculty members prepare students to solve problems and positively impact the world.

CPU times: user 343 ms, sys: 3.81 ms, total: 347 ms
Wall time: 346 ms


In [ ]:
%%time
# 125 50
row = qa_test[55]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What must students do to ensure accuracy before traveling with their visa?

Information:

```
Receive and Review Your Entry Visa: Check the visa in your passport to make sure that all the information on it is correct.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Check the visa in your passport to make sure that all the information on it is correct.
Finetuned model answer: Students must receive and review their entry visa in their passport to ensure that all information is correct.

CPU times: user 632 ms, sys: 0 ns, total: 632 ms
Wall time: 670 ms


In [ ]:
%%time
# 125 50
row = qa_test[98]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the purpose of the Bioengineering Opportunity Fund at the University of Kansas?

Information:

```
Bioengineering Opportunity Fund: Unrestricted funds that allow the Director to meet urgent needs and take advantage of opportunities as they arise.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     The Bioengineering Opportunity Fund provides unrestricted funds that allow the Director to meet urgent needs and take advantage of opportunities as they arise.
Finetuned model answer: The Bioengineering Opportunity Fund allows the Director to meet urgent needs and take advantage of opportunities as they arise.

CPU times: user 517 ms, sys: 2.64 ms, total: 519 ms
Wall time: 522 ms


In [ ]:
%%time
# 125 50
row = qa_test[12]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the process for submitting a large format printing request?

Information:

```
Request deadlines are 2:00pm Mondays and Wednesdays. (Poster is ready at 4pm Mon if submitted after 2pm Wed. Poster is ready at 4pm Wed if submitted before 2pm Wed.)
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Requests for printing must be submitted by 2:00 p.m. on Mondays and Wednesdays, with completion times depending on the submission timing.
Finetuned model answer: The process for submitting a large format printing request is as follows: the poster is ready at 4pm Mon if submitted after 2pm Wed., and at 4pm Wed if submitted before 2pm Wed.

CPU times: user 1.47 s, sys: 2.54 ms, total: 1.47 s
Wall time: 2.27 s


In [ ]:
%%time
# 125 50
row = qa_test[609]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

Which engineering discipline has students coming from different states?

Information:

```
Siana Gonzalez Aerospace Engineering, 2028 Hometown: Hialeah, FL Liam Mckinley Aerospace Engineering, 2028 Hometown: Olathe, KS
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Aerospace Engineering has students from Florida and Kansas.
Finetuned model answer: Aerospace Engineering.

CPU times: user 170 ms, sys: 2.06 ms, total: 173 ms
Wall time: 176 ms


In [ ]:
%%time
# 125 50
row = qa_test[21]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

Who will be speaking on March 10, 2025, and what is their title?

Information:

```
Monday: 03/10/2025 Iredia David Iyamu Assistant Professor Department of Medicinal Chemistry University of Kansas
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Iredia David Iyamu will be the speaker on March 10, 2025, and he is an Assistant Professor in the Department of Medicinal Chemistry.
Finetuned model answer: Iredia David Iyamu, Assistant Professor, Department of Medicinal Chemistry, University of Kansas.

CPU times: user 513 ms, sys: 0 ns, total: 513 ms
Wall time: 512 ms


In [ ]:
%%time
# 125 50
row = qa_test[803]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

How many credit hours of EECS classes must a Ph.D. student complete at a minimum?

Information:

```
Minimum of 30 credit hours of course work must be EECS classes numbered 700 or higher (or equivalent transferred classes), excluding Directed Graduate Reading (EECS 801), Graduate Problems (EECS 891), Master's Thesis (EECS 899), and Post-Master's Research (EECS 998).
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     A minimum of 30 credit hours of course work must be EECS classes numbered 700 or higher.
Finetuned model answer: 30 credit hours of EECS classes numbered 700 or higher.

CPU times: user 351 ms, sys: 690 µs, total: 352 ms
Wall time: 352 ms


In [ ]:
%%time
# 125 50
row = qa_test[456]
prompt = test_prompt(row)
print(prompt)
outputs = pipe(prompt)
response = f"""
Actual Answer:     {row["answer"]}
Finetuned model answer: {outputs[0]["generated_text"]}
"""
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

Which schools and centers contribute affiliate faculty to the bioengineering program at KU?

Information:

```
Our world-class affiliate faculty are found in the School of Medicine, School of Pharmacy, School of Business, College of Liberal Arts & Sciences, as well as in the many bioscience research centers and labs.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Actual Answer:     Affiliate faculty come from the School of Medicine, School of Pharmacy, School of Business, College of Liberal Arts & Sciences, and various bioscience research centers and labs.
Finetuned model answer: The affiliate faculty are found in the School of Medicine, School of Pharmacy, School of Business, College of Liberal Arts & Sciences, as well as in the many bioscience resear